In [1]:
from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

from models import WorkersOrm, ResumesOrm, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from pydantic import BaseModel

#### Без relationship

In [3]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True ) for row in result_orm] #from_attributes - обращение к атрибутам класса - конвертация к json
    print(f"{result_dto=}")

result_orm=[<WorkersOrm id=1,username=Bobr>, <WorkersOrm id=2,username=Misha>]
result_dto=[WorkersDTO(username='Bobr', id=1), WorkersDTO(username='Misha', id=2)]


#### C relationship

In [4]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .options(selectinload(WorkersOrm.resumes))
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm] # рекурсивная конвертация
    print(f"{result_dto=}")

result_orm=[<WorkersOrm id=1,username=Bobr>, <WorkersOrm id=2,username=Misha>]
result_dto=[WorkersRelDTO(username='Bobr', id=1, resumes=[ResumesDTO(title='Python Junior Developer', compensation=50000, workload=<Workload.fulltime: 'fulltime'>, worker_id=1, id=1, created_at=datetime.datetime(2024, 7, 26, 0, 48, 40), updated_at=datetime.datetime(2024, 7, 26, 0, 48, 40)), ResumesDTO(title='Python Разработчик', compensation=150000, workload=<Workload.fulltime: 'fulltime'>, worker_id=1, id=2, created_at=datetime.datetime(2024, 7, 26, 0, 48, 40), updated_at=datetime.datetime(2024, 7, 26, 0, 48, 40))]), WorkersRelDTO(username='Misha', id=2, resumes=[ResumesDTO(title='Python Data Engineer', compensation=250000, workload=<Workload.fulltime: 'fulltime'>, worker_id=2, id=3, created_at=datetime.datetime(2024, 7, 26, 0, 48, 40), updated_at=datetime.datetime(2024, 7, 26, 0, 48, 40)), ResumesDTO(title='Data Scientist', compensation=300000, workload=<Workload.fulltime: 'fulltime'>, worker_id=2, id=4, c

#### JOIN

In [5]:
class WorkloadAvgCompensationDTO(BaseModel):
    workload: Workload
    avg_compensation: int

In [8]:
with session_factory() as session:
    query = (
        select(
            ResumesOrm.workload,
            func.avg(ResumesOrm.compensation).cast(Integer).label("avg_compensation"),
        )
        .select_from(ResumesOrm)
        .filter(and_(
            ResumesOrm.title.contains("Python"),
            ResumesOrm.compensation > 40000,
        ))
        .group_by(ResumesOrm.workload)
        .having(func.avg(ResumesOrm.compensation) > 70000)
    )
    res = session.execute(query)
    result_orm = res.all()
    print(f"{result_orm=}")
    result_dto = [WorkloadAvgCompensationDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

result_orm=[('fulltime', 150000)]
result_dto=[WorkloadAvgCompensationDTO(workload=<Workload.fulltime: 'fulltime'>, avg_compensation=150000)]
